In [1]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from collections import Counter
from imblearn.under_sampling import NearMiss,CondensedNearestNeighbour, TomekLinks, EditedNearestNeighbours, OneSidedSelection, NeighbourhoodCleaningRule
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from matplotlib import pyplot
from numpy import where
import pandas as pd 
import os 
import json
import numpy as np
import pylab as plt
import tensorflow as tf
import time
from vecstack import stacking
import seaborn as sns
from math import floor

pd.set_option("display.max_columns", None)

In [2]:
# Read the datasets for the first scenario (07:00-09:00)
merged1 = pd.read_csv(os.path.join('./0709', 'merged1.csv')) #DoS
merged2 = pd.read_csv(os.path.join('./0709', 'merged2.csv')) #DoS Random
merged3 = pd.read_csv(os.path.join('./0709', 'merged3.csv')) #DoS Disruptive
merged4 = pd.read_csv(os.path.join('./0709', 'merged4.csv')) #DoS Random Sybil
merged5 = pd.read_csv(os.path.join('./0709', 'merged5.csv')) #DoS Disruptive Sybil

### Define sampling and classification functions

### Results per attack

In [21]:
def sampling_per_attack(sample,veremi):
    veremi =veremi[['pos_x_send', 'pos_y_send', 
       'spd_x_send', 'spd_y_send', 'acl_x_send', 'acl_y_send', 'hed_x_send', 'hed_y_send', 'label', 
       'nb_packets_sent', 'frequency1', 'frequency2', 'time_diff', 'distance', 'difSpeed', 'estAoA']]
    veremi.dropna(inplace=True)

    a = veremi.label.unique()[1]
    veremi['label'] = veremi['label'].replace(a,1)
    veremi = veremi.astype(np.uint8)
    
    if sample==0:
        # Data split (original dataset)
        y = veremi['label']
        X = veremi.drop(columns='label')
        
       
    elif sample==-1:
        # under sampling 
        u = veremi.label.value_counts().min()
        veremi_under_sampling = pd.concat([veremi[veremi['label']==0].sample(u), 
                                          veremi[veremi['label']==1].sample(u)], axis= 0)
        y = veremi_under_sampling['label']
        X = veremi_under_sampling.drop(columns='label')
        
        
    elif sample==1:
        # over sampling 
        o = veremi.label.value_counts().max()
        veremi_over_sampling = pd.concat([veremi[veremi['label']==0].sample(o, replace=True), 
                                          veremi[veremi['label']==1].sample(o, replace=True)], axis= 0)
        y = veremi_over_sampling['label']
        X = veremi_over_sampling.drop(columns='label')
        
        
    return(X,y)




In [22]:
def classification_per_attack(sample,method,veremi):
    X,y = sampling_per_attack(sample,veremi)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0, stratify=y)
             
    if method=='RF':
        # Random Forest
        rf = RandomForestClassifier(n_estimators=50)

        start = time.time()
        rf.fit(X_train, y_train)
        timefit = time.time() - start

        start = time.time()
        y_pred = rf.predict(X_test)
        timepred = time.time() - start
    
    elif method=='Xgboost':
        # XGBoost
        xgb = XGBClassifier()

        start = time.time()
        xgb.fit(X_train,y_train)
        timefit = time.time() - start

        start = time.time()
        y_pred = xgb.predict(X_test)
        timepred = time.time() - start

        
    elif method=='Stacking':
        estimators = [
            ('rf', RandomForestClassifier(n_estimators=50, criterion= 'gini')), #50, 300 or 400
            ('xgbc', XGBClassifier(eta=1, max_depth= 5, sampling_method= 'uniform')) #n_estimators=3000 or 5000
        ]
        
        
        from sklearn.ensemble import StackingClassifier
        from sklearn.linear_model import LogisticRegression
        
        clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

        start = time.time()
        clf.fit(X_train, y_train).score(X_test, y_test)
        timefit = time.time() - start

        start = time.time()
        y_pred = clf.predict(X_test)
        timepred = time.time() - start 
              
   
       
    report = classification_report(y_test,y_pred,output_dict=True)

    return(report['accuracy'], report['weighted avg']['precision'], report['weighted avg']['recall'], 
          report['weighted avg']['f1-score'], balanced_accuracy_score(y_test, y_pred),
          matthews_corrcoef(y_test, y_pred, sample_weight=None), timefit, timepred)


In [5]:
results_original_per_attack = pd.DataFrame(index=['Xgboost_1','RF_1','Stacking_1',
                                                 'Xgboost_2','RF_2','Stacking_2',
                                                 'Xgboost_3','RF_3','Stacking_3',
                                                 'Xgboost_4','RF_4','Stacking_4',
                                                 'Xgboost_5','RF_5','Stacking_5'],
                       columns=['Acc','Pre','Rec','F1s','BAS','MCC','TimeFit','TimePred'])

In [6]:
results_original_per_attack.xs('Xgboost_1')[:] = classification_per_attack(0,'Xgboost',merged1)
print(results_original_per_attack)
results_original_per_attack.xs('RF_1')[:] = classification_per_attack(0,'RF',merged1)
print(results_original_per_attack)
results_original_per_attack.xs('Stacking_1')[:] = classification_per_attack(0,'Stacking',merged1)
print(results_original_per_attack)
results_original_per_attack.to_excel('results_original_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_1       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [8]:
results_original_per_attack.xs('Xgboost_2')[:] = classification_per_attack(0,'Xgboost',merged2)
print(results_original_per_attack)
results_original_per_attack.xs('RF_2')[:] = classification_per_attack(0,'RF',merged2)
print(results_original_per_attack)
results_original_per_attack.xs('Stacking_2')[:] = classification_per_attack(0,'Stacking',merged2)
print(results_original_per_attack)
results_original_per_attack.to_excel('results_original_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [10]:
results_original_per_attack.xs('Xgboost_3')[:] = classification_per_attack(0,'Xgboost',merged3)
print(results_original_per_attack)
results_original_per_attack.xs('RF_3')[:] = classification_per_attack(0,'RF',merged3)
print(results_original_per_attack)
results_original_per_attack.xs('Stacking_3')[:] = classification_per_attack(0,'Stacking',merged3)
print(results_original_per_attack)
results_original_per_attack.to_excel('results_original_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [11]:
results_original_per_attack.xs('Xgboost_4')[:] = classification_per_attack(0,'Xgboost',merged4)
print(results_original_per_attack)
results_original_per_attack.xs('RF_4')[:] = classification_per_attack(0,'RF',merged4)
print(results_original_per_attack)
results_original_per_attack.xs('Stacking_4')[:] = classification_per_attack(0,'Stacking',merged4)
print(results_original_per_attack)
results_original_per_attack.to_excel('results_original_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [12]:
results_original_per_attack.xs('Xgboost_5')[:] = classification_per_attack(0,'Xgboost',merged5)
print(results_original_per_attack)
results_original_per_attack.xs('RF_5')[:] = classification_per_attack(0,'RF',merged5)
print(results_original_per_attack)
results_original_per_attack.xs('Stacking_5')[:] = classification_per_attack(0,'Stacking',merged5)
print(results_original_per_attack)
results_original_per_attack.to_excel('results_original_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921927  0.927738  0.921927  0.920786  0.909946  0.844329   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.996048  0.996061  0.996048   0.99605  0.996307  0.991921   
Stacking_1  0.996033  0.996043  0.996033  0.996034  0.996242  0.991886   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [5]:
results_under_per_attack = pd.DataFrame(index=['Xgboost_1','RF_1','Stacking_1',
                                                 'Xgboost_2','RF_2','Stacking_2',
                                                 'Xgboost_3','RF_3','Stacking_3',
                                                 'Xgboost_4','RF_4','Stacking_4',
                                                 'Xgboost_5','RF_5','Stacking_5'],
                       columns=['Acc','Pre','Rec','F1s','BAS','MCC','TimeFit','TimePred'])

In [6]:
results_under_per_attack.xs('Xgboost_1')[:] = classification_per_attack(-1,'Xgboost',merged1)
print(results_under_per_attack)
results_under_per_attack.xs('RF_1')[:] = classification_per_attack(-1,'RF',merged1)
print(results_under_per_attack)
results_under_per_attack.xs('Stacking_1')[:] = classification_per_attack(-1,'Stacking',merged1)
print(results_under_per_attack)
results_under_per_attack.to_excel('results_under_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_1       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [10]:
results_under_per_attack.xs('Xgboost_2')[:] = classification_per_attack(-1,'Xgboost',merged2)
print(results_under_per_attack)
results_under_per_attack.xs('RF_2')[:] = classification_per_attack(-1,'RF',merged2)
print(results_under_per_attack)
results_under_per_attack.xs('Stacking_2')[:] = classification_per_attack(-1,'Stacking',merged2)
print(results_under_per_attack)
results_under_per_attack.to_excel('results_under_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [13]:
results_under_per_attack.xs('Xgboost_3')[:] = classification_per_attack(-1,'Xgboost',merged3)
print(results_under_per_attack)
results_under_per_attack.xs('RF_3')[:] = classification_per_attack(-1,'RF',merged3)
print(results_under_per_attack)
results_under_per_attack.xs('Stacking_3')[:] = classification_per_attack(-1,'Stacking',merged3)
print(results_under_per_attack)
results_under_per_attack.to_excel('results_under_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [16]:
results_under_per_attack.xs('Xgboost_4')[:] = classification_per_attack(-1,'Xgboost',merged4)
print(results_under_per_attack)
results_under_per_attack.xs('RF_4')[:] = classification_per_attack(-1,'RF',merged4)
print(results_under_per_attack)
results_under_per_attack.xs('Stacking_4')[:] = classification_per_attack(-1,'Stacking',merged4)
print(results_under_per_attack)
results_under_per_attack.to_excel('results_under_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [19]:
results_under_per_attack.xs('Xgboost_5')[:] = classification_per_attack(-1,'Xgboost',merged5)
print(results_under_per_attack)
results_under_per_attack.xs('RF_5')[:] = classification_per_attack(-1,'RF',merged5)
print(results_under_per_attack)
results_under_per_attack.xs('Stacking_5')[:] = classification_per_attack(-1,'Stacking',merged5)
print(results_under_per_attack)
results_under_per_attack.to_excel('results_under_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.918448  0.924044  0.918448  0.918178  0.918448  0.842473   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1         0.99597  0.995982   0.99597   0.99597   0.99597  0.991952   
Stacking_1  0.996019  0.996027  0.996019  0.996019  0.996019  0.992047   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [7]:
results_over_per_attack = pd.DataFrame(index=['Xgboost_1','RF_1','Stacking_1',
                                                 'Xgboost_2','RF_2','Stacking_2',
                                                 'Xgboost_3','RF_3','Stacking_3',
                                                 'Xgboost_4','RF_4','Stacking_4',
                                                 'Xgboost_5','RF_5','Stacking_5'],
                       columns=['Acc','Pre','Rec','F1s','BAS','MCC','TimeFit','TimePred'])

In [8]:
results_over_per_attack.xs('Xgboost_1')[:] = classification_per_attack(1,'Xgboost',merged1)
print(results_over_per_attack)
results_over_per_attack.xs('RF_1')[:] = classification_per_attack(1,'RF',merged1)
print(results_over_per_attack)
results_over_per_attack.xs('Stacking_1')[:] = classification_per_attack(1,'Stacking',merged1)
print(results_over_per_attack)
results_over_per_attack.to_excel('results_over_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_1       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [11]:
results_over_per_attack.xs('Xgboost_2')[:] = classification_per_attack(1,'Xgboost',merged2)
print(results_over_per_attack)
results_over_per_attack.xs('RF_2')[:] = classification_per_attack(1,'RF',merged2)
print(results_over_per_attack)
results_over_per_attack.xs('Stacking_2')[:] = classification_per_attack(1,'Stacking',merged2)
print(results_over_per_attack)
results_over_per_attack.to_excel('results_over_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [14]:
results_over_per_attack.xs('Xgboost_3')[:] = classification_per_attack(1,'Xgboost',merged3)
print(results_over_per_attack)
results_over_per_attack.xs('RF_3')[:] = classification_per_attack(1,'RF',merged3)
print(results_over_per_attack)
results_over_per_attack.xs('Stacking_3')[:] = classification_per_attack(1,'Stacking',merged3)
print(results_over_per_attack)
results_over_per_attack.to_excel('results_over_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [17]:
results_over_per_attack.xs('Xgboost_4')[:] = classification_per_attack(1,'Xgboost',merged4)
print(results_over_per_attack)
results_over_per_attack.xs('RF_4')[:] = classification_per_attack(1,'RF',merged4)
print(results_over_per_attack)
results_over_per_attack.xs('Stacking_4')[:] = classification_per_attack(1,'Stacking',merged4)
print(results_over_per_attack)
results_over_per_attack.to_excel('results_over_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [20]:
results_over_per_attack.xs('Xgboost_5')[:] = classification_per_attack(1,'Xgboost',merged5)
print(results_over_per_attack)
results_over_per_attack.xs('RF_5')[:] = classification_per_attack(1,'RF',merged5)
print(results_over_per_attack)
results_over_per_attack.xs('Stacking_5')[:] = classification_per_attack(1,'Stacking',merged5)
print(results_over_per_attack)
results_over_per_attack.to_excel('results_over_per_attack_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.921569  0.927089  0.921569  0.921315  0.921569  0.848641   
MLP_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_1            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_1         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_1           NaN       NaN       NaN       NaN       NaN       NaN   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.998095  0.998097  0.998095  0.998095  0.998095  0.996192   
Stacking_1  0.998129  0.998131  0.998129  0.998129  0.998129   0.99626   
Xgboost_2        1.0       1.0       1.0       1.0       1.0       1.0   
MLP_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_2            NaN       NaN       NaN       NaN       NaN       NaN   
DBN_BO_2         NaN       NaN       NaN       NaN       NaN       NaN   
LSTM_2           NaN       NaN       N

In [ ]:
# Read the results if needed
results_original_per_attack = pd.read_excel('results_original_per_attack_1.xlsx')
results_original_per_attack.rename(columns={'Unnamed: 0': 'method'}, inplace=True)
results_original_per_attack.set_index('method', inplace=True)

results_under_per_attack = pd.read_excel('results_under_per_attack_1.xlsx')
results_under_per_attack.rename(columns={'Unnamed: 0': 'method'}, inplace=True)
results_under_per_attack.set_index('method', inplace=True)

results_over_per_attack = pd.read_excel('results_over_per_attack_1.xlsx')
results_over_per_attack.rename(columns={'Unnamed: 0': 'method'}, inplace=True)
results_over_per_attack.set_index('method', inplace=True)

#### Results for the main dataset (not adding new features)

In [26]:
def sampling_per_attack_original(sample,veremi):
    veremi =veremi[['pos_x_send', 'pos_y_send',
       'pos_noise_x', 'pos_noise_y', 'spd_x_send', 'spd_y_send', 'spd_noise_x',
       'spd_noise_y', 'acl_x_send', 'acl_y_send', 'acl_noise_x', 'acl_noise_y',
       'hed_x_send', 'hed_y_send', 'hed_noise_x', 'hed_noise_y', 'label']]
    veremi.dropna(inplace=True)

    a = veremi.label.unique()[1]
    veremi['label'] = veremi['label'].replace(a,1)
    veremi = veremi.astype(np.uint8)
    
    if sample==0:
        # Data split (original dataset)
        y = veremi['label']
        X = veremi.drop(columns='label')
      
    
       
    elif sample==-1:
        # under sampling 
        u = veremi.label.value_counts().min()
        veremi_under_sampling = pd.concat([veremi[veremi['label']==0].sample(u), 
                                          veremi[veremi['label']==1].sample(u)], axis= 0)
        y = veremi_under_sampling['label']
        X = veremi_under_sampling.drop(columns='label')
       
        
    elif sample==1:
        # over sampling 
        o = veremi.label.value_counts().max()
        veremi_over_sampling = pd.concat([veremi[veremi['label']==0].sample(o, replace=True), 
                                          veremi[veremi['label']==1].sample(o, replace=True)], axis= 0)
        y = veremi_over_sampling['label']
        X = veremi_over_sampling.drop(columns='label')
   
        
    return(X,y)




In [27]:
def classification_per_attack_original(sample,method,veremi):
    X,y = sampling_per_attack_original(sample,veremi)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0, stratify=y)
    
    if method=='RF':
        # Random Forest
        rf = RandomForestClassifier(n_estimators=50)

        start = time.time()
        rf.fit(X_train, y_train)
        timefit = time.time() - start

        start = time.time()
        y_pred = rf.predict(X_test)
        timepred = time.time() - start
    
    elif method=='Xgboost':
        # XGBoost
        xgb = XGBClassifier()

        start = time.time()
        xgb.fit(X_train,y_train)
        timefit = time.time() - start

        start = time.time()
        y_pred = xgb.predict(X_test)
        timepred = time.time() - start

        
    elif method=='Stacking':
        estimators = [
            ('rf', RandomForestClassifier(n_estimators=50, criterion= 'gini')), #50, 300 or 400
            ('xgbc', XGBClassifier(eta=1, max_depth= 5, sampling_method= 'uniform')) #n_estimators=3000 or 5000
        ]
        
        
        from sklearn.ensemble import StackingClassifier
        from sklearn.linear_model import LogisticRegression
        
        clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())

        start = time.time()
        clf.fit(X_train, y_train).score(X_test, y_test)
        timefit = time.time() - start

        start = time.time()
        y_pred = clf.predict(X_test)
        timepred = time.time() - start 
        
 
        
    report = classification_report(y_test,y_pred,output_dict=True)

    return(report['accuracy'], report['weighted avg']['precision'], report['weighted avg']['recall'], 
          report['weighted avg']['f1-score'], balanced_accuracy_score(y_test, y_pred),
          matthews_corrcoef(y_test, y_pred, sample_weight=None), timefit, timepred)


In [28]:
results_original_per_attack_original = pd.DataFrame(index=['Xgboost_1','RF_1','Stacking_1',
                                                 'Xgboost_2','RF_2','Stacking_2',
                                                 'Xgboost_3','RF_3','Stacking_3',
                                                 'Xgboost_4','RF_4','Stacking_4',
                                                 'Xgboost_5','RF_5','Stacking_5'],
                       columns=['Acc','Pre','Rec','F1s','BAS','MCC','TimeFit','TimePred'])

In [29]:
results_original_per_attack_original.xs('Xgboost_1')[:] = classification_per_attack_original(0,'Xgboost',merged1)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('RF_1')[:] = classification_per_attack_original(0,'RF',merged1)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('Stacking_1')[:] = classification_per_attack_original(0,'Stacking',merged1)
print(results_original_per_attack_original)
results_original_per_attack_original.to_excel('results_original_per_attack_original_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.758046  0.761913  0.758046  0.752087  0.739532  0.503528   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_1       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_2        NaN       NaN       NaN       NaN       NaN       NaN   
kNN_2            NaN       NaN       NaN       NaN       NaN       NaN   
RF_2             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_2       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_3        NaN       NaN       NaN       NaN       NaN       NaN   
kNN_3            NaN       NaN       NaN       NaN       NaN       NaN   
RF_3             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_3       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_4        NaN       NaN       N

In [30]:
results_original_per_attack_original.xs('Xgboost_2')[:] = classification_per_attack_original(0,'Xgboost',merged2)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('RF_2')[:] = classification_per_attack_original(0,'RF',merged2)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('Stacking_2')[:] = classification_per_attack_original(0,'Stacking',merged2)
print(results_original_per_attack_original)
results_original_per_attack_original.to_excel('results_original_per_attack_original_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.758046  0.761913  0.758046  0.752087  0.739532  0.503528   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.990856  0.990885  0.990856   0.99085  0.990051  0.981375   
Stacking_1   0.99099  0.991002   0.99099  0.990986  0.990379  0.981636   
Xgboost_2   0.998442  0.998444  0.998442  0.998443  0.998493  0.996864   
kNN_2            NaN       NaN       NaN       NaN       NaN       NaN   
RF_2             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_2       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_3        NaN       NaN       NaN       NaN       NaN       NaN   
kNN_3            NaN       NaN       NaN       NaN       NaN       NaN   
RF_3             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_3       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_4        NaN       NaN       N

In [31]:
results_original_per_attack_original.xs('Xgboost_3')[:] = classification_per_attack_original(0,'Xgboost',merged3)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('RF_3')[:] = classification_per_attack_original(0,'RF',merged3)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('Stacking_3')[:] = classification_per_attack_original(0,'Stacking',merged3)
print(results_original_per_attack_original)
results_original_per_attack_original.to_excel('results_original_per_attack_original_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.758046  0.761913  0.758046  0.752087  0.739532  0.503528   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.990856  0.990885  0.990856   0.99085  0.990051  0.981375   
Stacking_1   0.99099  0.991002   0.99099  0.990986  0.990379  0.981636   
Xgboost_2   0.998442  0.998444  0.998442  0.998443  0.998493  0.996864   
kNN_2            NaN       NaN       NaN       NaN       NaN       NaN   
RF_2        0.999911  0.999911  0.999911  0.999911  0.999907  0.999821   
Stacking_2   0.99993   0.99993   0.99993   0.99993  0.999928  0.999859   
Xgboost_3   0.615832   0.61422  0.615832  0.598683  0.594577  0.207378   
kNN_3            NaN       NaN       NaN       NaN       NaN       NaN   
RF_3             NaN       NaN       NaN       NaN       NaN       NaN   
Stacking_3       NaN       NaN       NaN       NaN       NaN       NaN   
Xgboost_4        NaN       NaN       N

In [32]:
results_original_per_attack_original.xs('Xgboost_4')[:] = classification_per_attack_original(0,'Xgboost',merged4)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('RF_4')[:] = classification_per_attack_original(0,'RF',merged4)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('Stacking_4')[:] = classification_per_attack_original(0,'Stacking',merged4)
print(results_original_per_attack_original)
results_original_per_attack_original.to_excel('results_original_per_attack_original_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.758046  0.761913  0.758046  0.752087  0.739532  0.503528   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.990856  0.990885  0.990856   0.99085  0.990051  0.981375   
Stacking_1   0.99099  0.991002   0.99099  0.990986  0.990379  0.981636   
Xgboost_2   0.998442  0.998444  0.998442  0.998443  0.998493  0.996864   
kNN_2            NaN       NaN       NaN       NaN       NaN       NaN   
RF_2        0.999911  0.999911  0.999911  0.999911  0.999907  0.999821   
Stacking_2   0.99993   0.99993   0.99993   0.99993  0.999928  0.999859   
Xgboost_3   0.615832   0.61422  0.615832  0.598683  0.594577  0.207378   
kNN_3            NaN       NaN       NaN       NaN       NaN       NaN   
RF_3        0.752931  0.757016  0.752931  0.748249  0.740266  0.499271   
Stacking_3  0.752126  0.753158  0.752126  0.749191  0.742174  0.496013   
Xgboost_4   0.999293  0.999293  0.9992

In [33]:
results_original_per_attack_original.xs('Xgboost_5')[:] = classification_per_attack_original(0,'Xgboost',merged5)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('RF_5')[:] = classification_per_attack_original(0,'RF',merged5)
print(results_original_per_attack_original)
results_original_per_attack_original.xs('Stacking_5')[:] = classification_per_attack_original(0,'Stacking',merged5)
print(results_original_per_attack_original)
results_original_per_attack_original.to_excel('results_original_per_attack_original_1.xlsx')

                 Acc       Pre       Rec       F1s       BAS       MCC  \
Xgboost_1   0.758046  0.761913  0.758046  0.752087  0.739532  0.503528   
kNN_1            NaN       NaN       NaN       NaN       NaN       NaN   
RF_1        0.990856  0.990885  0.990856   0.99085  0.990051  0.981375   
Stacking_1   0.99099  0.991002   0.99099  0.990986  0.990379  0.981636   
Xgboost_2   0.998442  0.998444  0.998442  0.998443  0.998493  0.996864   
kNN_2            NaN       NaN       NaN       NaN       NaN       NaN   
RF_2        0.999911  0.999911  0.999911  0.999911  0.999907  0.999821   
Stacking_2   0.99993   0.99993   0.99993   0.99993  0.999928  0.999859   
Xgboost_3   0.615832   0.61422  0.615832  0.598683  0.594577  0.207378   
kNN_3            NaN       NaN       NaN       NaN       NaN       NaN   
RF_3        0.752931  0.757016  0.752931  0.748249  0.740266  0.499271   
Stacking_3  0.752126  0.753158  0.752126  0.749191  0.742174  0.496013   
Xgboost_4   0.999293  0.999293  0.9992

In [ ]:
# Read the results if needed
results_original_per_attack_original = pd.read_excel('results_original_per_attack_original_1.xlsx')
results_original_per_attack_original.rename(columns={'Unnamed: 0': 'method'}, inplace=True)
results_original_per_attack_original.set_index('method', inplace=True)